<a href="https://colab.research.google.com/github/Vj-Ydv/trump_tweets_analysis/blob/master/Aspect_based_sentiment_analysis_SC_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy

In [2]:
# Importing the required libraries
import spacy
sp = spacy.load("en_core_web_sm")
from textblob import TextBlob

# Creating a list of positive and negative sentences.
mixed_sen = [
'This chocolate truffle cake is really tasty',
'This party is amazing!',
'My mom is the best!',
'App response is very slow!'
'The trip to US was very boring',
'The tour to Nepal was very enjoyable',
'Web app works slow and reliable results with high accuracy'
]

# An empty list for obtaining the extracted aspects
# from sentences.
ext_aspects = []

# Performing Aspect Extraction
for sen in mixed_sen:
  important = sp(sen)
  descriptive_item = ''
  target = ''
  for token in important:
	  if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
	    target = token.text
	  if token.pos_ == 'ADJ':
	    added_terms = ''
	    for mini_token in token.children:
		    if mini_token.pos_ != 'ADV':
		      continue
		    added_terms += mini_token.text + ' '
	    descriptive_item = added_terms + token.text 
  ext_aspects.append({'aspect': target,'description': descriptive_item})

print("ASPECT EXTRACTION\n")
print(ext_aspects)


for aspect in ext_aspects:
  aspect['sentiment'] = TextBlob(aspect['description']).sentiment

print("\n")
print("SENTIMENT ASSOCIATION\n")
print(ext_aspects)


ASPECT EXTRACTION

[{'aspect': 'cake', 'description': 'really tasty'}, {'aspect': 'party', 'description': 'amazing'}, {'aspect': 'mom', 'description': 'best'}, {'aspect': 'trip', 'description': 'very boring'}, {'aspect': 'tour', 'description': 'very enjoyable'}, {'aspect': 'app', 'description': 'high'}]


SENTIMENT ASSOCIATION

[{'aspect': 'cake', 'description': 'really tasty', 'sentiment': Sentiment(polarity=0.2, subjectivity=0.2)}, {'aspect': 'party', 'description': 'amazing', 'sentiment': Sentiment(polarity=0.6000000000000001, subjectivity=0.9)}, {'aspect': 'mom', 'description': 'best', 'sentiment': Sentiment(polarity=1.0, subjectivity=0.3)}, {'aspect': 'trip', 'description': 'very boring', 'sentiment': Sentiment(polarity=-1.0, subjectivity=1.0)}, {'aspect': 'tour', 'description': 'very enjoyable', 'sentiment': Sentiment(polarity=0.65, subjectivity=0.78)}, {'aspect': 'app', 'description': 'high', 'sentiment': Sentiment(polarity=0.16, subjectivity=0.5399999999999999)}]


In [ ]:
!pip install pandas
!pip install numpy
!pip install nltk
!pip install stanfordnlp

     |████████████████████████████████| 158 kB 8.8 MB/s 


In [ ]:
!pip install stanza

     |████████████████████████████████| 432 kB 11.3 MB/s 
     |████████████████████████████████| 174 kB 63.7 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=04062feca8deddc1f89b3de0d5a19afa01c3235ae3fa0ae2fba70896764b68b8
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
import stanfordnlp
import stanza

# Make sure you have downloaded the StanfordNLP English model and other essential tools using,
#stanfordnlp.download('en')
stanza.download('en')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def aspect_sentiment_analysis(txt, stop_words, nlp):
    
    txt = txt.lower() # LowerCasing the given Text
    sentList = nltk.sent_tokenize(txt) # Splitting the text into sentences

    fcluster = []
    totalfeatureList = []
    finalcluster = []
    dic = {}

    for line in sentList:
        newtaggedList = []
        txt_list = nltk.word_tokenize(line) # Splitting up into words
        taggedList = nltk.pos_tag(txt_list) # Doing Part-of-Speech Tagging to each word

        newwordList = []
        flag = 0
        for i in range(0,len(taggedList)-1):
            if(taggedList[i][1]=="NN" and taggedList[i+1][1]=="NN"): # If two consecutive words are Nouns then they are joined together
                newwordList.append(taggedList[i][0]+taggedList[i+1][0])
                flag=1
            else:
                if(flag==1):
                    flag=0
                    continue
                newwordList.append(taggedList[i][0])
                if(i==len(taggedList)-2):
                    newwordList.append(taggedList[i+1][0])

        finaltxt = ' '.join(word for word in newwordList) 
        new_txt_list = nltk.word_tokenize(finaltxt)
        wordsList = [w for w in new_txt_list if not w in stop_words]
        taggedList = nltk.pos_tag(wordsList)

        doc = nlp(finaltxt) # Object of Stanford NLP Pipeleine
        
        # Getting the dependency relations betwwen the words
        dep_node = []
        for dep_edge in doc.sentences[0].dependencies:
            dep_node.append([dep_edge[2].text, dep_edge[0].index, dep_edge[1]])

        # Coverting it into appropriate format
        for i in range(0, len(dep_node)):
            if (int(dep_node[i][1]) != 0):
                dep_node[i][1] = newwordList[(int(dep_node[i][1]) - 1)]

        featureList = []
        categories = []
        for i in taggedList:
            if(i[1]=='JJ' or i[1]=='NN' or i[1]=='JJR' or i[1]=='NNS' or i[1]=='RB'):
                featureList.append(list(i)) # For features for each sentence
                totalfeatureList.append(list(i)) # Stores the features of all the sentences in the text
                categories.append(i[0])

        for i in featureList:
            filist = []
            for j in dep_node:
                if((j[0]==i[0] or j[1]==i[0]) and (j[2] in ["nsubj", "acl:relcl", "obj", "dobj", "agent", "advmod", "amod", "neg", "prep_of", "acomp", "xcomp", "compound"])):
                    if(j[0]==i[0]):
                        filist.append(j[1])
                    else:
                        filist.append(j[0])
            fcluster.append([i[0], filist])
            
    for i in totalfeatureList:
        dic[i[0]] = i[1]
    
    for i in fcluster:
        if(dic[i[0]]=="NN"):
            finalcluster.append(i)
        
    return(finalcluster)




2022-02-11 08:19:59 INFO: Downloading default packages for language: en (English)...


2022-02-11 08:20:13 INFO: Finished downloading models and saved to /root/stanza_resources.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
nlp = stanza.Pipeline('en')
stop_words = set(stopwords.words('english'))
txt = "The Sound Quality is great but the battery life is very bad."
print(aspect_sentiment_analysis(txt, stop_words, nlp))



2022-02-11 08:21:58 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-02-11 08:21:59 INFO: Use device: cpu
2022-02-11 08:21:59 INFO: Loading: tokenize
2022-02-11 08:21:59 INFO: Loading: pos
2022-02-11 08:21:59 INFO: Loading: lemma
2022-02-11 08:21:59 INFO: Loading: depparse
2022-02-11 08:22:00 INFO: Loading: sentiment
2022-02-11 08:22:00 INFO: Loading: constituency
2022-02-11 08:22:01 INFO: Loading: ner
2022-02-11 08:22:02 INFO: Done loading processors!


AttributeError: ignored